In [1]:
from explicit import get_neutron_flux, get_neutron_flux_error
from settings import ureg
from data import foil_data
import pandas as pd
import pint_pandas

20231109_BABY-4_Nb-2I_0659_4hr
flux: (5.22 +/- 0.46)e+08 1 / second
error: 8.81%

20231109_BABY-4_Nb-2I_1623_4hr
flux: (5.35 +/- 0.47)e+08 1 / second
error: 8.81%

20231109_BABY-4_Nb-2J_2027_4hr
flux: (1.97 +/- 0.17)e+08 1 / second
error: 8.82%

20231110_BABY-4_Nb-2J_1442_4hr
flux: (2.12 +/- 0.19)e+08 1 / second
error: 8.82%

20231209_BABY-5_Nb-B_A125dt_0922_4hr
flux: (2.31 +/- 0.20)e+08 1 / second
error: 8.82%

20231209_BABY-5_Nb-B_A125dt_1609_4hr
flux: (2.35 +/- 0.21)e+08 1 / second
error: 8.82%

20231208_BABY-5_Nb-D_P383dt_1132_4hr
flux: (5.36 +/- 0.47)e+08 1 / second
error: 8.81%

20231208_BABY-5_Nb-D_P383dt_1555_4hr
flux: (5.48 +/- 0.48)e+08 1 / second
error: 8.81%

20240128_BABY6_Nb-A_A-325_1054_4hr
flux: (2.12 +/- 0.19)e+08 1 / second
error: 8.82%

20240128_BABY6_Nb-A_A-325_1638_4hr
flux: (2.14 +/- 0.19)e+08 1 / second
error: 8.82%

20240127_BABY6_Nb-C_P383_1313_4hr
flux: (4.98 +/- 0.44)e+08 1 / second
error: 8.81%

20240127_BABY6_Nb-C_P383_1738_4hr
flux: (4.93 +/- 0.43)e+08 1 /

In [11]:
# make a pandas dataframe based on the foil data with columns for run, generator, flux and error

irradiations = [
    {"t_on": 0, "t_off": 12 * ureg.h},
    {"t_on": 24 * ureg.h, "t_off": 36 * ureg.h},
]

data = {
    "run": [],
    "generator": [],
    "flux": [],
    "error": [],

}
for name, foil in foil_data.items():
    flux = get_neutron_flux(foil, irradiations)
    error = get_neutron_flux_error(foil)
    data["run"].append(foil["run"])
    data["generator"].append(foil["generator"])
    data["flux"].append(flux.to(ureg.s**-1))
    data["error"].append(error)
data["flux"] = pd.Series(data["flux"], dtype="pint[s**-1]")
pd.options.display.float_format = '{:,.2e}'.format
df = pd.DataFrame(data)
df.pint.dequantify()

,run,generator,flux,error
unit,No Unit,No Unit,1 / second,No Unit
0,4,P383,5.22e+08,8.81e-02
1,4,P383,5.35e+08,8.81e-02
2,4,A325,1.97e+08,8.82e-02
3,4,A325,2.12e+08,8.82e-02
4,5,A325,2.31e+08,8.82e-02
5,5,A325,2.35e+08,8.82e-02
6,5,P383,5.36e+08,8.81e-02
7,5,P383,5.48e+08,8.81e-02
8,6,A325,2.12e+08,8.82e-02


In [13]:
#for each run number compute the average per generator
grouped_multiple = df.groupby(['run', 'generator']).agg({'flux': ['mean', 'min', 'max']})
grouped_multiple.columns = ['flux_mean', 'flux_min', 'flux_max']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple.pint.dequantify()


,run,generator,flux_mean,flux_min,flux_max
unit,No Unit,No Unit,1 / second,1 / second,1 / second
0,4,A325,2.05e+08,1.97e+08,2.12e+08
1,4,P383,5.29e+08,5.22e+08,5.35e+08
2,5,A325,2.33e+08,2.31e+08,2.35e+08
3,5,P383,5.42e+08,5.36e+08,5.48e+08
4,6,A325,2.13e+08,2.12e+08,2.14e+08
5,6,P383,4.95e+08,4.93e+08,4.98e+08
